In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StructType
from pyspark.sql.functions import lit

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/26 23:13:28 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/26 23:13:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/26 23:13:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/26 23:13:28 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
rankSchema = StructType().add("_c0", "string").add("_c1", "double")
df = spark.readStream.option('sep', '\t').schema(rankSchema).csv('gs://as-systems-hw2/whole')


# Select the devices which have signal more than 10
filter_rank = df.select("*").where("_c1 > 0.5").coalesce(1)#.withColumn('key',lit(1)).groupBy('key').count()



In [3]:
# Start running the query that prints the running counts to the console
query = filter_rank \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option('checkpointLocation','gs://as-systems-hw2/checkpoint') \
    .start('gs://as-systems-hw2/output')

#query.awaitTermination()

22/04/26 23:13:39 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
